In [15]:
!pip install scipy

     |████████████████████████████████| 25.9 MB 29.1 MB/s eta 0:00:01
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential
import numpy as np

import datetime, os

In [3]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
def model_builder(output_size):
    model = Sequential()
    model.add(Conv2D(input_shape=(32, 32,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    
    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=output_size, activation="softmax"))
    
    return model

In [5]:
## test the model with dummy data
num_classes = 10
net = model_builder(num_classes)
net.summary()
inp = tf.random.normal(shape=(1, 32, 32, 3))
out = net(inp)
print(out.shape)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 8, 256)         2

In [6]:
def normalize_inputs(inputs, regl):
  ## using mean/stdv normalization
  mean = np.mean(inputs, axis=(0, 1, 2, 3))
  stdv = np.std(inputs, axis=(0, 1, 2, 3))

  normalized_inputs = (inputs - mean) / (stdv + regl)
  return normalized_inputs

In [7]:
## train the model
## training parameter
batch_size = 128
epochs = 5#250
learning_rate = 0.001
lr_decay = 1e-6
lr_drop = 20
regl = 1e-7

In [8]:
## load the dataset
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train
y_train = y_train

In [9]:
## normalize the data
x_train = normalize_inputs(x_train, regl)
x_test = normalize_inputs(x_test, regl)

In [10]:
num_classes = np.max(y_train) + 1
#num_classes
#y_train
## encode the labels
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
#y_train[1]

In [11]:
## Augment the data
augmentor = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,  
    height_shift_range=0.1, 
    horizontal_flip=True,
    vertical_flip=False
)
augmentor.fit(x_train)

In [12]:
## model optimization
## sgd optimizer
sgd = tf.keras.optimizers.SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)
## loss function is cross entropy loss and performance metric is accuracy
net.compile(loss='categorical_crossentropy', optimizer=sgd,metrics=['accuracy'])
#print(*next(augmentor.flow(x_train, y_train, batch_size)))

In [13]:
## define tensorboard keras callbacks
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [14]:
## train the model
net.fit(
    augmentor.flow(x_train, y_train, batch_size),
    #x=x_train, y=y_train,
    #steps_per_epoch=x_train[0] // batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test), 
    #verbose=1,
    batch_size=batch_size,
    workers=4,
    callbacks=[tensorboard_callback]
)

net.save_weights('cifar10vgg.h5')

Epoch 1/5
  1/391 [..............................] - ETA: 0s - loss: 2.3026 - accuracy: 0.0938WARNING:tensorflow:From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
391/391 [==============================] - 16s 42ms/step - loss: 2.3025 - accuracy: 0.1062 - val_loss: 2.3023 - val_accuracy: 0.1084
Epoch 2/5
391/391 [==============================] - 15s 39ms/step - loss: 2.3023 - accuracy: 0.1237 - val_loss: 2.3021 - val_accuracy: 0.1204
Epoch 3/5
391/391 [==============================] - 15s 39ms/step - loss: 2.3020 - accuracy: 0.1230 - val_loss: 2.3018 - val_accuracy: 0.1745
Epoch 4/5
391/391 [==============================] - 15s 39ms/step - loss: 2.3017 - accuracy: 0.1363 - val_loss: 2.3013 - val_accuracy: 0.1942
Epoch 5/5
391/391 [==============================] - 15s 39ms/step

In [18]:
%tensorboard --logdir logs

In [17]:
!kill 330